```
Copyright (c) 2024, Google Inc.
All rights reserved.
Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.
3. Neither the name of Google Inc. nor the names of its contributors
   may be used to endorse or promote products derived from this software without
   specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON
ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```


# Imports

In [1]:
import concurrent.futures
import os
import random

import google.auth
import google.auth.transport.requests
import numpy as np


# Authentication

The JSON file mentioned in the cell below is created by running the following command (for service accounts)

```
gcloud auth application-default login --impersonate-service-account SERVICE_ACCT
```

or that command

```
gcloud auth application-default login
```

to identify with your own account.

This assumes that you have first [installed](https://cloud.google.com/sdk/docs/install) `gcloud` CLI and created a service account (see [[1]](https://cloud.google.com/iam/docs/service-account-overview), [[2]](https://cloud.google.com/iam/docs/service-accounts-create)) (identified by `SERVICE_ACCT` above)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/to/your/credentials/json/file'

In [2]:
# Environment variable `GOOGLE_APPLICATION_CREDENTIALS` must be set for these
# imports to work.
import api_utils

# Online predictions

## With raw audio

In [ ]:
raw_audio = np.array([[random.random() for _ in range(32000)] for _ in range(4)])
embeddings = api_utils.make_prediction(
  endpoint_path=api_utils.RAW_AUDIO_ENDPOINT_PATH,
  instances=raw_audio,
)

## With GCS bucket URIs

In [ ]:
gcs_creds, project = google.auth.default()
api_utils.initial_token_refresh(gcs_creds)

In [ ]:
# copybara:strip_begin(Internal repo)
gcs_bucket_name = 'hear-demo'
# copybara:strip_end_and_replace_begin
# gcs_bucket_name = 'your-bucket-name'
# copybara:replace_end

predictions = api_utils.make_prediction(
  endpoint_path=api_utils.GCS_URI_ENDPOINT_PATH,
  # copybara:strip_begin(Internal filepaths)
  instances=['data/test.wav', 'data/test.wav'],
  # copybara:strip_end_and_replace_begin
  # instances=['path/to/your/file1.wav', 'path/to/your/file2.wav'],
  # copybara:replace_end
  gcs_bucket_name=gcs_bucket_name,
  gcs_creds=gcs_creds,
)

# If you have a lot of queries to run

Example with the raw-audio endpoint (202) using ThreadPoolExecutor.

In [ ]:
# 1000 batches of 4 clips. This is the format expected for the raw audio endpoint
instances = np.random.uniform(size=(1000, 4, 32000))  # update with your data

responses = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
  futures_to_batch_idx = {
    executor.submit(
        api_utils.make_prediction_with_exponential_backoff,
        api_utils.RAW_AUDIO_ENDPOINT_PATH,
        instance
    ): batch_idx
    for batch_idx, instance in enumerate(instances)
  }

  for future in concurrent.futures.as_completed(futures_to_batch_idx):
    batch_idx = futures_to_batch_idx[future]
    try:
      responses[batch_idx] = future.result()
    except Exception as e:
      print("An error occurred:", e)